## RSI练习

(a) 自行获取股票和日期    
(b) 计算6日RSI、30日RSI的值，并绘制出两者的曲线图；    
(c) 设定6日RSI取值大于90,为超买区，释放出卖出信号；6日RSI取值小于10时，为超卖区，释放出买入信号。编写Python代码计算这种买卖信号预测的准确率；    
(d) 当6日RSI线向上穿过30日RSI线时，为“黄金交叉”,释放出买入信号；当6日RSI线向下穿过30日RSI线时，为“死亡交叉”,释放出卖出信号。根据长短线的“黄金交叉”与“死亡交叉”策略，编写Python代码实现这一策略，并进行回测；

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tushare as ts

ts.set_token('')
ts.pro_api()
ticker=''
start_date=''
end_date=''
data=ts.get_k_data(ticker,start_data,end_date)
data.set_index("data",implace='True')
data.index=pd.to_datatime(data.index)
# 请确保数据包含日期、收盘价字段，并按日期升序排列

# (b) 计算6日RSI、30日RSI的值，并绘制出两者的曲线图
def calculate_rsi(data, window):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

data['6_day_rsi'] = calculate_rsi(data, 6)
data['30_day_rsi'] = calculate_rsi(data, 30)

plt.figure(figsize=(12, 6))
plt.plot(data['Date'], data['6_day_rsi'], label='6-day RSI')
plt.plot(data['Date'], data['30_day_rsi'], label='30-day RSI')
plt.legend()
plt.show()

# (c) 计算买卖信号预测的准确率
data['buy_signal'] = np.where(data['6_day_rsi'] < 10, 1, 0)
data['sell_signal'] = np.where(data['6_day_rsi'] > 90, 1, 0)

# 假设actual_buy_signal和actual_sell_signal是实际的买卖信号
accuracy_buy = (data['buy_signal'] == actual_buy_signal).mean()
accuracy_sell = (data['sell_signal'] == actual_sell_signal).mean()

print(f'买入信号准确率：{accuracy_buy}')
print(f'卖出信号准确率：{accuracy_sell}')

# (d) 实现黄金交叉和死亡交叉策略的回测
data['signal'] = 0
data['signal'][1:] = np.where((data['6_day_rsi'].shift(1) < data['30_day_rsi'].shift(1)) & (data['6_day_rsi'] > data['30_day_rsi']), 1, 0)
data['signal'][1:] = np.where((data['6_day_rsi'].shift(1) > data['30_day_rsi'].shift(1)) & (data['6_day_rsi'] < data['30_day_rsi']), -1, data['signal'][1:])

data['return'] = data['Close'].pct_change() * data['signal'].shift(1)
total_return = data['return'].sum()

print(f'总收益率：{total_return}')